In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn.functional as F

In [2]:
im_frame = Image.open('data/Dataset_BUSI_with_GT/benign/benign (300)_mask.png')
np_frame = np.array(im_frame)

In [3]:
np_frame.shape

(387, 463)

In [67]:
pred = torch.randn(size=(2, 19, 1024, 2048))

In [68]:
def iou_loss(pred, target, num_classes = 19):
  pred = F.softmax(pred, dim = 1) # chuyển về thành xác suất cho từng channel
  iou_per_class = []
  for c in range(num_classes):
    pred_c = pred[:,c,:,:] # (B, H, W) cho channel c đang xét
    target_c = (target == c).float() # trả về 1.0 if index có giá trị là c else 0.0
    intersection = torch.sum(pred_c * target_c, dim = (1,2)) # tính giao
    union = torch.sum(pred_c, dim = (1,2)) + torch.sum(target_c, dim = (1,2)) - intersection # tính hợp
    iou = torch.where(union == 0, torch.tensor(1.0).to(pred.device), intersection / union)  # tính iou, nếu union == 0, gán iou = 1
    iou_per_class.append(iou)
    # có thể tính thêm dice loss = 2 * intersection / union + intersection
    
  mean_iou = torch.mean(torch.stack(iou_per_class, dim = 1), dim = 1) # stack iou theo từng ảnh rồi tính mean iou cho từng ảnh
  iou_loss = 1 - mean_iou
  final_loss = torch.mean(iou_loss) # mean iou loss 
  return final_loss

In [69]:
def calculate_iou(output, target, num_classes=19):
    with torch.no_grad():
        # Lấy nhãn dự đoán bằng cách lấy argmax theo chiều classes
        predicted = torch.argmax(output, dim=1)  # [batch_size, height, width]

        # Tạo tensor để lưu trữ tổng IoU cho từng lớp
        ious = []

        # Lấy các lớp xuất hiện trong predicted và target
        classes_in_true = torch.unique(target)
        classes_in_pred = torch.unique(predicted)

        # Loại bỏ lớp -1 khỏi danh sách các lớp cần duyệt trong ground truth
        classes_in_true_no_neg1 = classes_in_true[classes_in_true != -1]

        # Tính IoU cho các lớp có trong ground truth (ngoại trừ lớp -1)
        for cls in classes_in_true_no_neg1:
            cls = cls.item()  # Chuyển sang kiểu int để sử dụng như chỉ mục

            # Mask cho từng lớp
            pred_mask = (predicted == cls)
            true_mask = (target == cls)

            # Tính Intersection và Union
            intersection = torch.sum(pred_mask & true_mask).item()
            union = torch.sum(pred_mask | true_mask).item()

            if union == 0:
                ious.append(float('nan'))  # Nếu không có pixel nào thuộc lớp đó
            else:
                ious.append(intersection / union)

        # Tính IoU cho lớp -1 trong `y_true` với các lớp trong `y_pred` mà không có trong `y_true`
        neg1_pixels = (target == -1)
        if torch.sum(neg1_pixels) > 0:
            pred_classes_not_in_true = torch.tensor(
                [cls for cls in classes_in_pred if cls not in classes_in_true_no_neg1],
                device=target.device
            )

            # Tính IoU cho các pixel lớp -1
            pred_mask_neg1 = torch.isin(predicted, pred_classes_not_in_true)  # Chỉ có trên PyTorch 1.10+
            intersection_neg1 = torch.sum(neg1_pixels & pred_mask_neg1).item()
            union_neg1 = torch.sum(neg1_pixels | pred_mask_neg1).item()

            if union_neg1 == 0:
                ious.append(float('nan'))
            else:
                ious.append(intersection_neg1 / union_neg1)

    # Tính mIoU bằng cách lấy trung bình các IoU, bỏ qua giá trị NaN
    return np.nanmean(ious)

In [70]:
iou_loss(pred, target)

tensor(0.9730)

In [71]:
calculate_iou(pred, target, num_classes=19)

0.02698446676252961

In [72]:
predicted = torch.argmax(pred, dim=1)